## Here we use Roberta which has already been pretrained using 58 million tweets. If the prediction is 0-> Negative 1->Neutral 2->Positive

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


## a tweeet can consists of @mention, tweet, emojis and links 

In [2]:
tweet = "@RenjithAnil I got good grades @VIT 😊 https://vit.ac.in/"

# Preprocessing the tweet so as to feed it in appropriate format to the pretrained model

In [3]:
tweet_words = []

for word in tweet.split(' '):
    if word.startswith('@') and len(word):
        word = '@user'
    elif word.startswith('https'):
        word = "https"
    elif word.startswith('http'):
        word = "http"
    tweet_words.append(word)
    
print(tweet_words)

['@user', 'I', 'got', 'good', 'grades', '@user', '😊', 'https']


In [4]:
tweet_proc = " ".join(tweet_words)
tweet_proc

'@user I got good grades @user 😊 https'

## Downloading the pretrained model

### url: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [5]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
tokenizer = AutoTokenizer.from_pretrained(roberta)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [7]:
labels = ['Negative','Neutral','Positive']

## Sentiment Analysis

In [8]:
encoded_tweet = tokenizer(tweet_proc,return_tensors = 'pt')
print(encoded_tweet)

{'input_ids': tensor([[    0,  1039, 12105,    38,   300,   205, 11165,   787, 12105, 17841,
         27969,  1205,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


## Passing encoded tweet to model

In [9]:
output = model(**encoded_tweet)

In [10]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-2.9504,  0.0589,  3.3391]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
scores = output[0][0].detach().numpy()
print(scores)

[-2.9504018   0.05890664  3.3391435 ]


In [12]:
scores = softmax(scores) # Converting into numbers between 0 and 1
scores

array([0.00178514, 0.03619071, 0.96202415], dtype=float32)

In [13]:
maxim = max(scores)
for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    if s==maxim:
        final_label = l
    print(l,s)

Negative 0.0017851357
Neutral 0.036190715
Positive 0.96202415


In [14]:
print("The tweet is:",final_label)

The tweet is: Positive
